## RNN
**자연어처리, 음성인식처럼 순서있는 데이터 처리하는데 강점 가진 신경망<br/>**
( 앞이나 뒤의 정보에 따라 전체 의미가 달라지거나, 앞의 정보로 다음에 나올 정보 추측하려는 경우에 RNN 사용하면 성능 좋음 )
- 셀을 여러개 중첩하여 심층 신경망 만듦
- 앞단계에서 학습한 결과 다음단계의 학습에 이용<br/>
→ 학습 데이터를 단계별로 구분하여 입력해야 함

In [1]:
import tensorflow as tf

from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("./mnist/data/", one_hot=True)

learning_rate = 0.001
total_epoch = 30
batch_size = 128

# RNN 은 순서가 있는 자료를 다루므로, 한 번에 입력받는 갯수와, 총 몇 단계로 이뤄진 데이터를 받을지 설정해야함
# 가로 픽셀수를 n_input 으로, 세로 픽셀수를 n_step 으로 설정
n_input = 28
n_step = 28
n_hidden = 128
n_class = 10

X = tf.placeholder(tf.float32, [None, n_step, n_input])
Y = tf.placeholder(tf.float32, [None, n_class])

W = tf.Variable(tf.random_normal([n_hidden, n_class]))
b = tf.Variable(tf.random_normal([n_class]))

Extracting ./mnist/data/train-images-idx3-ubyte.gz
Extracting ./mnist/data/train-labels-idx1-ubyte.gz
Extracting ./mnist/data/t10k-images-idx3-ubyte.gz
Extracting ./mnist/data/t10k-labels-idx1-ubyte.gz


In [2]:
# n_hidden개의 출력값 갖는 RNN 셀 생성
cell = tf.nn.rnn_cell.BasicRNNCell(n_hidden)

In [3]:
# RNN 신경망 완성
outputs, states = tf.nn.dynamic_rnn(cell, X, dtype=tf.float32)

In [4]:
# RNN 신경망에서 나오는 출력값은 [batch_size, n_step, n_hidden]
# 최종 결과값은 실측값과 동일한 형태인 [batch_size, n_class]이어야함
# 은닉층의 출력값 형태를 변경해야, W와 행렬곱 수행해서 원하는 최종 출력값 형태 얻을 수 있음
# -> [n_step, batch_size, n_hidden]
outputs = tf.transpose(outputs, [1, 0, 2])
# -> [batch_size, n_hidden]
outputs = outputs[-1]

In [5]:
model = tf.matmul(outputs, W) + b

In [6]:
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=model, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)

In [7]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

total_batch = int(mnist.train.num_examples/batch_size)

for epoch in range(total_epoch):
    total_cost = 0

    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        # RNN 입력값 형태에 맞게 [batch_size, n_step, n_input] 형태로 바꿔줌
        batch_xs = batch_xs.reshape((batch_size, n_step, n_input))

        _, cost_val = sess.run([optimizer, cost],
                               feed_dict={X: batch_xs, Y: batch_ys})
        total_cost += cost_val

    print('Epoch:', '%04d' % (epoch + 1),
          'Avg. cost =', '{:.3f}'.format(total_cost / total_batch))

print('최적화 완료!')

is_correct = tf.equal(tf.argmax(model, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))

test_batch_size = len(mnist.test.images)
test_xs = mnist.test.images.reshape(test_batch_size, n_step, n_input)
test_ys = mnist.test.labels

print('정확도:', sess.run(accuracy,
                       feed_dict={X: test_xs, Y: test_ys}))

Epoch: 0001 Avg. cost = 0.568
Epoch: 0002 Avg. cost = 0.246
Epoch: 0003 Avg. cost = 0.191
Epoch: 0004 Avg. cost = 0.154
Epoch: 0005 Avg. cost = 0.143
Epoch: 0006 Avg. cost = 0.127
Epoch: 0007 Avg. cost = 0.117
Epoch: 0008 Avg. cost = 0.109
Epoch: 0009 Avg. cost = 0.107
Epoch: 0010 Avg. cost = 0.098
Epoch: 0011 Avg. cost = 0.096
Epoch: 0012 Avg. cost = 0.093
Epoch: 0013 Avg. cost = 0.089
Epoch: 0014 Avg. cost = 0.088
Epoch: 0015 Avg. cost = 0.086
Epoch: 0016 Avg. cost = 0.077
Epoch: 0017 Avg. cost = 0.077
Epoch: 0018 Avg. cost = 0.076
Epoch: 0019 Avg. cost = 0.079
Epoch: 0020 Avg. cost = 0.073
Epoch: 0021 Avg. cost = 0.068
Epoch: 0022 Avg. cost = 0.075
Epoch: 0023 Avg. cost = 0.067
Epoch: 0024 Avg. cost = 0.063
Epoch: 0025 Avg. cost = 0.063
Epoch: 0026 Avg. cost = 0.066
Epoch: 0027 Avg. cost = 0.067
Epoch: 0028 Avg. cost = 0.061
Epoch: 0029 Avg. cost = 0.068
Epoch: 0030 Avg. cost = 0.062
최적화 완료!
정확도: 0.9789


**RNN 모델을 이용하여 단어를 자동 완성하는 프로그램**
- 영문자 4개로 구성된 단어를 학습시켜, 3글자만 주어지면 나머지 한 글자를 추천하여 단어를 완성하는 프로그램

In [1]:
import tensorflow as tf
import numpy as np

char_arr = ['a', 'b', 'c', 'd', 'e', 'f', 'g',
            'h', 'i', 'j', 'k', 'l', 'm', 'n',
            'o', 'p', 'q', 'r', 's', 't', 'u',
            'v', 'w', 'x', 'y', 'z']

# 원-핫 인코딩 사용 및 디코딩 위하여 연관 배열 만듦
# {'a': 0, 'b': 1, 'c': 2, ..., 'j': 9, 'k', 10, ...}
num_dic = {n: i for i, n in enumerate(char_arr)}
dic_len = len(num_dic)

In [2]:
# 학습에 사용할 단어
seq_data = ['word', 'wood', 'deep', 'dive', 'cold', 'cool', 'load', 'love', 'kiss', 'kind']

In [3]:
# 단어들을 학습에 사용할 수 있는 형식으로 변환해주는 유틸리티 함수

def make_batch(seq_data):
    input_batch = []
    target_batch = []
    
    for seq in seq_data:
        # 입력값용
        # 단어의 처음 세글자 알파벳 인덱스를 구한 배열 만듦
        input = [num_dic[n] for n in seq[:-1]]
        # 출력값용
        # 마지막 글자의 알파벳 인덱스를 구함
        target = num_dic[seq[-1]]
        # 입력값을 원-핫 인코딩으로 변환
        input_batch.append(np.eye(dic_len)[input])
        # 손실함수로 sparse_softmax_cross_entropy_with_logits 사용할 것이기 때문에
        # labels 값에 원-핫 인코딩 사용하지 않고 그대로 넘겨줘도 됨
        # 그동안 사용했던 손실함수인 softmax_cross_entropy_with_logits 는 label 값을 원-핫 인코딩으로 넘겨줘야 함
        target_batch.append(target)
        
    return input_batch, target_batch

In [4]:
learning_rate = 0.01
n_hidden = 128
total_epoch = 30

n_step = 3
# sparse_softmax_cross_entropy_with_logits 를 사용할 때
# 실측값인 labels 값은 인덱스의 숫자 그대로 사용하고
# 예측모델의 출력값은 인덱스의 원-핫 인코딩을 사용해야 함
# -> n_class 값도 n_input과 마찬가지로 dic_len 과 크기 같음
n_input = n_class = dic_len

In [7]:
X = tf.placeholder(tf.float32, [None, n_step, n_input])
Y = tf.placeholder(tf.int32, [None])

W = tf.Variable(tf.random_normal([n_hidden, n_class]))
b = tf.Variable(tf.random_normal([n_class]))

In [8]:
# 두개의 RNN 셀 -> 심층 신경망
cell1 = tf.nn.rnn_cell.BasicLSTMCell(n_hidden)
# 과적합 방지를 위한 드롭아웃 기법 적용
cell1 = tf.nn.rnn_cell.DropoutWrapper(cell1, output_keep_prob=0.5)
cell2 = tf.nn.rnn_cell.BasicLSTMCell(n_hidden)

In [10]:
# 두개의 셀 조합
multi_cell = tf.nn.rnn_cell.MultiRNNCell([cell1, cell2])
outputs, states = tf.nn.dynamic_rnn(multi_cell, X, dtype=tf.float32)

In [11]:
outputs = tf.transpose(outputs, [1, 0, 2])
outputs = outputs[-1]
model = tf.matmul(outputs, W) + b

In [12]:
cost = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits=model, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)

In [13]:
# 신경망 학습

sess = tf.Session()
sess.run(tf.global_variables_initializer())

input_batch, target_batch = make_batch(seq_data)

for epoch in range(total_epoch):
    _, loss = sess.run([optimizer, cost], feed_dict={X: input_batch, Y: target_batch})

    print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.6f}'.format(loss))

print('최적화 완료!')

Epoch: 0001 cost = 3.487972
Epoch: 0002 cost = 2.411238
Epoch: 0003 cost = 1.702744
Epoch: 0004 cost = 1.056619
Epoch: 0005 cost = 1.073420
Epoch: 0006 cost = 0.583005
Epoch: 0007 cost = 0.345159
Epoch: 0008 cost = 0.571403
Epoch: 0009 cost = 0.483696
Epoch: 0010 cost = 0.205288
Epoch: 0011 cost = 0.336313
Epoch: 0012 cost = 0.229422
Epoch: 0013 cost = 0.171057
Epoch: 0014 cost = 0.180321
Epoch: 0015 cost = 0.061512
Epoch: 0016 cost = 0.071990
Epoch: 0017 cost = 0.122550
Epoch: 0018 cost = 0.151526
Epoch: 0019 cost = 0.075986
Epoch: 0020 cost = 0.054566
Epoch: 0021 cost = 0.064611
Epoch: 0022 cost = 0.021537
Epoch: 0023 cost = 0.089278
Epoch: 0024 cost = 0.012444
Epoch: 0025 cost = 0.077762
Epoch: 0026 cost = 0.020067
Epoch: 0027 cost = 0.005534
Epoch: 0028 cost = 0.007483
Epoch: 0029 cost = 0.026209
Epoch: 0030 cost = 0.000735
최적화 완료!


In [14]:
# 실측값 Y는 인덱스를 그대로 사용하므로 정수이다
# -> Y와의 비교를 위해 argmax로 변환한 예측값도 정수로 변경 해주어야함
prediction = tf.cast(tf.argmax(model, 1), tf.int32)
prediction_check = tf.equal(prediction, Y)
accuracy = tf.reduce_mean(tf.cast(prediction_check, tf.float32))

In [15]:
input_batch, target_batch = make_batch(seq_data)

predict, accuracy_val = sess.run([prediction, accuracy], feed_dict={X: input_batch, Y: target_batch})

In [16]:
predict_words = []
for idx, val in enumerate(seq_data):
    last_char = char_arr[predict[idx]]
    predict_words.append(val[:3] + last_char)

print('\n=== 예측 결과 ===')
print('입력값:', [w[:3] + ' ' for w in seq_data])
print('예측값:', predict_words)
print('정확도:', accuracy_val)


=== 예측 결과 ===
입력값: ['wor ', 'woo ', 'dee ', 'div ', 'col ', 'coo ', 'loa ', 'lov ', 'kis ', 'kin ']
예측값: ['word', 'wood', 'deep', 'dive', 'cold', 'cool', 'load', 'love', 'kiss', 'kind']
정확도: 1.0


### Sequence to Sequence
- 순차적 정보 입력받는 신경망(RNN)과 출력하는 신경망 조합한 모델
- 입력을 위한 신경망인 인코더와 출력을 위한 신경망인 디코더
- 구글이 기계번역에 사용하는 신경망 모델
- 번역,챗봇 등 문장 입력받아 다른 문장 출력하는 프로그램에서 사용

>**특수한 심볼<br/>**
    > S : 디코더에 입력이 시작됨을 알려주는 심볼<br/>
    > E : 디코더의 출력이 끝났음을 알려주는 심볼<br/>
    > P : 빈 데이터를 채울 때 사용하는 아무 의미없는 심볼

In [1]:
#####
# 네 글자의 영어 단어를 입력받아 두 글자의 한글 단어로 번역하는 프로그램
#####

import tensorflow as tf
import numpy as np

char_arr = [c for c in 'SEPabcdefghijklmnopqrstuvwxyz단어나무놀이소녀키스사랑']
num_dic = {n: i for i, n in enumerate(char_arr)}
dic_len = len(num_dic)

seq_data = [['word', '단어'], ['wood', '나무'],
            ['game', '놀이'], ['girl', '소녀'],
            ['kiss', '키스'], ['love', '사랑']]

In [2]:
# 입력 단어와 출력 단어를 한 글자씩 떼어낸 뒤 배열로 만든 후에
# 원-핫 인코딩 형식으로까지 만들어주는 유틸리티 함수

def make_batch(seq_data):
    input_batch = []
    output_batch = []
    target_batch = []

    for seq in seq_data:
        # 인코더 셀의 입력값
        # 입력단어의 글자들을 한글자씩 떼어 배열로 만듦
        input = [num_dic[n] for n in seq[0]]
        # 디코더 셀의 입력값
        # 시작을 나타내는 심볼 S를 맨 앞에 붙임
        output = [num_dic[n] for n in ('S' + seq[1])]
        # 학습을 위해 비교할 디코더 셀의 출력값
        # 출력의 끝을 알려주는 심볼 E를 마지막에 붙임
        target = [num_dic[n] for n in (seq[1] + 'E')]

        input_batch.append(np.eye(dic_len)[input])
        output_batch.append(np.eye(dic_len)[output])
        # 실측값인 디코더 셀의 출력값은 원-핫 인코딩이 아닌 인덱스 숫자 그대로 사용
        target_batch.append(target)

    return input_batch, output_batch, target_batch

In [3]:
learning_rate = 0.01
n_hidden = 128
total_epoch = 100
n_class = n_input = dic_len

enc_input = tf.placeholder(tf.float32, [None, None, n_input])
dec_input = tf.placeholder(tf.float32, [None, None, n_input])
targets = tf.placeholder(tf.int64, [None, None])

In [4]:
# 인코더 셀
with tf.variable_scope('encode'):
    enc_cell = tf.nn.rnn_cell.BasicRNNCell(n_hidden)
    enc_cell = tf.nn.rnn_cell.DropoutWrapper(enc_cell, output_keep_prob=0.5)
    
    outputs, enc_states = tf.nn.dynamic_rnn(enc_cell, enc_input, dtype=tf.float32)

# 디코더 셀   
with tf.variable_scope('decode'):
    dec_cell = tf.nn.rnn_cell.BasicRNNCell(n_hidden)
    dec_cell = tf.nn.rnn_cell.DropoutWrapper(dec_cell, output_keep_prob=0.5)
    # 디코더를 만들 때 초기 상태값(입력값X)으로 인코더의 최종 상태값 넣어줘야함
    outputs, dec_states = tf.nn.dynamic_rnn(dec_cell, dec_input, initial_state=enc_states, dtype=tf.float32)

In [5]:
model = tf.layers.dense(outputs, n_class, activation=None)

cost = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits=model, labels=targets))

optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)

In [6]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

input_batch, output_batch, target_batch = make_batch(seq_data)

for epoch in range(total_epoch):
    _, loss = sess.run([optimizer, cost], 
                       feed_dict={enc_input: input_batch, 
                                  dec_input: output_batch,
                                  targets: target_batch})

    print('Epoch:', '%04d' % (epoch + 1),
          'cost =', '{:.6f}'.format(loss))

print('최적화 완료!')

Epoch: 0001 cost = 3.890341
Epoch: 0002 cost = 2.884977
Epoch: 0003 cost = 1.737166
Epoch: 0004 cost = 1.350623
Epoch: 0005 cost = 0.825565
Epoch: 0006 cost = 0.629649
Epoch: 0007 cost = 0.458868
Epoch: 0008 cost = 0.235998
Epoch: 0009 cost = 0.332930
Epoch: 0010 cost = 0.163457
Epoch: 0011 cost = 0.082249
Epoch: 0012 cost = 0.144275
Epoch: 0013 cost = 0.111416
Epoch: 0014 cost = 0.099161
Epoch: 0015 cost = 0.110924
Epoch: 0016 cost = 0.117621
Epoch: 0017 cost = 0.048042
Epoch: 0018 cost = 0.161530
Epoch: 0019 cost = 0.027336
Epoch: 0020 cost = 0.073911
Epoch: 0021 cost = 0.009534
Epoch: 0022 cost = 0.065020
Epoch: 0023 cost = 0.047422
Epoch: 0024 cost = 0.005929
Epoch: 0025 cost = 0.021642
Epoch: 0026 cost = 0.005044
Epoch: 0027 cost = 0.014960
Epoch: 0028 cost = 0.007402
Epoch: 0029 cost = 0.022530
Epoch: 0030 cost = 0.012174
Epoch: 0031 cost = 0.025575
Epoch: 0032 cost = 0.009598
Epoch: 0033 cost = 0.004245
Epoch: 0034 cost = 0.004368
Epoch: 0035 cost = 0.003378
Epoch: 0036 cost = 0

In [7]:
def translate(word):
    # 이 모델은 입력값과 출력값 데이터로 [영어단어, 한글단어] 사용하지만,
    # 예측 시에는 한글단어를 알지 못하므로, 디코더의 입출력값 P 값으로 채운다.
    seq_data = [word, 'P' * len(word)]

    input_batch, output_batch, target_batch = make_batch([seq_data])

    prediction = tf.argmax(model, 2)

    result = sess.run(prediction,feed_dict={enc_input: input_batch,
                                            dec_input: output_batch,
                                            targets: target_batch})

    # 예측 결과는 글자의 인덱스를 뜻하는 숫자이므로,
    # 각 숫자에 해당하는 글자를 가져와 배열 만듦
    decoded = [char_arr[i] for i in result[0]]
    
    # 출력의 끝을 의미하는 E 이후의 글자들 제거하고 문자열로 
    end = decoded.index('E')
    translated = ''.join(decoded[:end])

    return translated

In [8]:
print('\n=== 번역 테스트 ===')

print('word ->', translate('word'))
print('wodr ->', translate('wodr'))
print('love ->', translate('love'))
print('loev ->', translate('loev'))
print('abcd ->', translate('abcd'))


=== 번역 테스트 ===
word -> 단어
wodr -> 나무
love -> 사랑
loev -> 사랑
abcd -> 소녀녀
